In [1]:
from NumbaMinpack import lmdif, minpack_sig
from numba import njit, cfunc
import numpy as np
from scipy.optimize import root

In [2]:
@cfunc(minpack_sig)
def myfunc(x, fvec, args):
    fvec[0] = x[0]**2 - 30.0
    fvec[1] = x[1]**2 - 8.0

funcptr = myfunc.address 
    
@njit
def myfunc_scipy(x):
    return np.array([x[0]**2 - 30.0,
                     x[1]**2 - 8.0])

In [3]:
x_init = np.array([10.0,10.0])
neqs = 2
args = np.array([0.0])

In [5]:
sol = lmdif(funcptr, x_init, neqs, args)
sol_sp = root(myfunc_scipy,x_init,method='lm')

print(sol)
print()
print(sol_sp)

(array([5.47722558, 2.82842712]), array([0.00000000e+00, 1.77635684e-15]), True, 2)

   cov_x: array([[ 0.00833333, -0.        ],
       [ 0.        ,  0.03125   ]])
    fjac: array([[-10.95445128,   0.        ],
       [  0.        ,  -5.65685433]])
     fun: array([0.00000000e+00, 1.77635684e-15])
    ipvt: array([1, 2], dtype=int32)
 message: 'The relative error between two consecutive iterates is at most 0.000000'
    nfev: 22
     qtf: array([ 0.00000000e+00, -3.55271368e-15])
  status: 2
 success: True
       x: array([5.47722558, 2.82842712])


In [9]:
%timeit lmdif(funcptr, x_init, neqs, args)
%timeit root(myfunc_scipy,x_init,method='lm')

6.08 µs ± 89.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
160 µs ± 884 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
# NumbaMinpack within jit compiled function works
@njit
def test():
    sol = lmdif(funcptr, x_init, neqs, args)
    return sol
test()

(array([5.47722558, 2.82842712]),
 array([0.00000000e+00, 1.77635684e-15]),
 True,
 2)

In [12]:
# scipy within jit compiled function does not work
@njit
def test_sp():
    sol_sp = root(myfunc_scipy,x_init,method='lm')
    return sol_sp
test_sp()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'root':[0m [1m[1mcannot determine Numba type of <class 'function'>[0m
[1m
File "<ipython-input-12-9e33a68959a9>", line 4:[0m
[1mdef test_sp():
[1m    sol_sp = root(myfunc_scipy,x_init,method='lm')
[0m    [1m^[0m[0m
[0m